# ADSP 32018: Final Project
## Exploratory Data Analysis and Data Cleaning

Peyton Nash

### Project Description
In March of 2023, Goldman Sachs published a report, indicating that ~25% of the tasks in US and Europe can be automated using AI.  However, not all industries will be affected equally. According to the report, certain jobs, like office tasks, legal, architecture, and social sciences have a potential for 30%+ automation, while positions like construction, installation, and building maintenance are going to be largely unaffected.

In July of 2025, Microsoft published an in-depth studyLinks to an external site. based on 200,000 anonymized conversations with Microsoft Copilot, aiming to understand how generative AI is actually being used in the workplace and which professions are being most affected.

The researchers separated what users intended to do from what the AI actually delivered. They then mapped both to detailed job functions defined by O*NET. Using this framework, along with indicators of task success and coverage, they developed an “AI applicability score” for every occupation.

The findings are clear. Generative AI excels at tasks like information gathering, writing, and communication. It is already transforming knowledge and service-based roles. However, it has limited usefulness in jobs that rely on physical effort.
One of the most surprising insights? There’s little connection between AI’s impact and factors like income or education level. This challenges long-held assumptions about which roles are most at risk of disruption.

You can also find supporting evidence in the Facebook Research paper, which highlights Moravec’s Paradox. This thesis posits that the hardest problems in AI involve sensorimotor skills rather than abstract thought or reasoning. Notably, these findings coincide with predictions made by Goldman Sachs.

For this final project, I have prepared a collection of ~200K news articles on our favorite topics, data science, machine learning, and artificial intelligence. Your task is to identify what industries are going to be most impacted by AI over the next several years, based on the information/insights you can extract from this text corpus.

Your goal is to provide actionable recommendations on what can be done with AI to automate the jobs, improve employee productivity, and generally make AI adoption successful. Please pay attention to the introduction of novel technologies and algorithms, such as AI for image generation and Conversational AI, as they represent the entire paradigm shift in adoption of AI technologies and data science in general.

### Setup

In [36]:
# Import libraries
import re, math, gc, itertools, warnings, os, random
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from pandarallel import pandarallel

import tldextract
from nltk import download
from nltk.corpus import words
from nltk.tokenize import sent_tokenize


from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

from datasketch import MinHash, MinHashLSH

In [2]:
# Read environment variables
load_dotenv()

True

In [3]:
# Load data
df = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')

# Check dimensions
print(f'Number of articles: {df.shape[0]}')

Number of articles: 200760


### Initial Data Checking

In [4]:
# Check head
df.head()

,url,date,language,title,text
0,http://businessnewsthisweek.com/business/infog...,2023-05-20,en,Infogain AI Business Solutions Now Available i...,\n\nInfogain AI Business Solutions Now Availab...
1,http://www.huewire.com/how-you-should-validate...,2023-07-21,en,How You Should Validate Machine Learning Model...,\n\nHow You Should Validate Machine Learning M...
2,http://www.huewire.com/vise-intelligence-is-a-...,2023-09-29,en,Vise Intelligence is a new AI to assist — not ...,\n\nVise Intelligence is a new AI to assist — ...
3,https://abcnews.go.com/Technology/google-makes...,2024-06-03,en,Google makes adjustments to AI Overviews after...,\n\nGoogle makes adjustments to AI Overviews a...
4,https://betanews.com/2023/06/08/wordpress-ai-a...,2023-06-08,en,WordPress' AI Assistant can write blog posts...,\n\n\n WordPress' AI Assistant can write blog...


In [5]:
# Check language of articles
print(df.groupby('language')['language'].count())

language
en    200760
Name: language, dtype: int64


In [6]:
# Check scrape data of articles
print(f'Earliest scrape data: {df["date"].min()}')
print(f'Latest scrape data: {df["date"].max()}')

Earliest scrape data: 2022-01-01
Latest scrape data: 2025-07-22


In [7]:
# Check the number of sources
df['domain'] = df['url'].apply(lambda x: tldextract.extract(x).domain)
print(f'Number of unique domains: {df["domain"].unique().shape[0]}\n')

# Check most common sources
source_count = df.groupby('domain')['domain'].count().sort_values(ascending=False)
print(f'Most common sources:\n {source_count[:10]}\n')
print(f'Number of sources with more than 100 articles: {source_count[source_count>100].shape[0]}\n')
print(f'Percentage of articles published by sources with more than 100 articles: {100 * source_count[source_count>100].sum()/source_count.sum():.2f}%\n')

Number of unique domains: 5119

Most common sources:
 domain
rawpixel        8831
citylife        4040
menafn          3847
einpresswire    3637
indiatimes      3594
prnewswire      3393
nasdaq          2473
yahoo           1982
levels          1576
livemint        1479
Name: domain, dtype: int64

Number of sources with more than 100 articles: 394

Percentage of articles published by sources with more than 100 articles: 70.60%



### Data Cleaning

In [ ]:
# Remove large irrelevant domains
domain_drop = ['rawpixel', 'levels', 'mexc']
df = df[~df['domain'].isin(domain_drop)]

print(f'Number of remaining articles: {len(df)}')

#### Remove Webscrape Remnants

In [8]:
# Identify domains so that articles are not grouped based on domain name
download('words')

words_eng = set(words.words())
domains = list(df['domain'].sort_values().unique())
domains_clean = []

for item in domains:
    if item.lower() not in words_eng:
        domains_clean.append(item)

del words, domains

[nltk_data] Downloading package words to
[nltk_data]     /Users/peytonnash/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [9]:
# Create list of common boilerplate text
boilerplate_patterns = [
    r" subscribe[.\s]| newsletter[.\s]| cookie policy[.\s]| privacy policy[.\s]| terms of service[.\s]",
    r" all rights reserved| copyright\s{1,3}[0-9]{4}[.\s]",
    r" advertisement[.\s]| sponsored[.\s]| sponsored content[.\s]| promoted[.\s]| story continues below[.\s]",
    r" most read[.\s]| most popular[.\s]| trending[.\s]| read more[.\s]| you might also like[.\s]| sitemap[.\s]| breaking news[.\s]",
    r" share this[.\s]| follow us[.\s]| contact us[.\s]| sign up[.\s]| log in[.\s]| register[.\s]| share on facebook[.\s]",
]

# r'subscribe | newsletter | cookie policy | privacy policy | terms of service | all rights reserved | copyright\s{1,3}[0-9]{4} | advertisement |sponsored | sponsored content |promoted | story continues below | most read | most popular | trending | read more | you might also like | sitemap | share this | follow us | contact us | sign up | log in | register |\n|\d{1,2}\:\d{1,2}|[.:-]| / '

# Combine the boiler plate text to split on it below
split_pattern = '|'.join(boilerplate_patterns) + '|\n|\d{1,2}\:\d{1,2}|\d{2}[-/]\d{2}[-/]\d{4}| / '

In [17]:
# Define a function to clean up the web-scrape remnants
def clean_text(text: str):
    if not isinstance(text, str) or not text.strip():
        return ""

    # Remove urls/emails
    text = re.sub(r"http[s]?://\S+|www\.\S+|\S+@\S+\.\S+", " ", text)

    # Remove non-English characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Normalize bullets/long dashes
    text = text.replace("•", " ").replace("–", "-").replace("—", "-")

    # Split the text into lines for line-level filtering
    lines = [line.strip() for line in re.split(split_pattern, text, flags=re.IGNORECASE) if line]

    pruned = []
    for ln in lines:
        if not ln:
            continue
        # Discard short lines with a high proportion of non-alphabetical characters
        if len(ln) < 30 and sum(c.isalpha() for c in ln) < 20:
            continue
        # Strip all-caps headlines unless reasonably long
        if ln.isupper() and len(ln) < 80:
            continue
        # Identify lines with more than 40% of the characters are non-alphabetical
        alpha = sum(c.isalpha() for c in ln)
        if alpha == 0 or alpha / max(len(ln), 1) < 0.4:
            continue
        if len(sent_tokenize(ln, language='english')) < 2:
            continue
        pruned.append(ln)

    # Combine the lines into a single text
    text = " ".join(pruned)

    # Remove most punctuation except intra-word hyphens/apostrophes
    text = re.sub(r"[^\w\s'\-$%.,:;/]", ' ', text)

    # Remove common webscrape remnants
    text = re.sub(split_pattern, ' ', text, flags=re.IGNORECASE)

    # Remove domain names
    domain_pattern = r"(?:\s(?:{})(?=\s))".format("|".join(domains_clean))
    text = re.sub(domain_pattern, ' ', text, re.IGNORECASE)

    # Reduce repeated whitespace to single space
    text = re.sub(r"\s+", " ", text).strip()

    # Remove text in brackets
    text = re.sub(r"\[[^\]]{0,60}\]", " ", text)
    text = re.sub(r"\([^)<>]{0,60}\)", " ", text)
    text = re.sub(r"\s{2,}", " ", text).strip()

    return text

def is_english(text: str) -> bool:
    try:
        if len(text) < 200:
            return True
        return detect(text) == "en"
    except Exception:
        return True

In [18]:
# Identify body text
pandarallel.initialize(progress_bar=True)

df['text_clean'] = df['text'].parallel_apply(clean_text)

INFO: Pandarallel will run on 11 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [19]:
# Identify English text
pandarallel.initialize(progress_bar=True)

df['english'] = df['text_clean'].parallel_apply(is_english)

INFO: Pandarallel will run on 11 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [20]:
print(f'Number of documents: {len(df)}')
print(f'Number of documents with non-empty text: {len(df[df["text_clean"] != ""])}')
print(f'Number of English documents: {len(df[df["english"]==True])}')

Number of documents: 200760
Number of documents with non-empty text: 197620
Number of English documents: 200575


In [21]:
df.to_parquet('output_data/df_clean.parquet')

In [22]:
# Get the length of the cleaned text
pandarallel.initialize(progress_bar=True)
df['text_length'] = df['text_clean'].parallel_apply(lambda x: len(x))

print(f'Length range: {df["text_length"].min()} to {df["text_length"].max()}')

# Remove the top and bottom five percent longest and shortest texts
pctl = np.percentile(df[df['text_length'] > 0]['text_length'], [.15, .95])
df = df[(df['text_length']<=pctl[0]) | (df['text_length']>=pctl[1])]

print(f'Number of articles after removing long and short articles: {len(df)}')
print(f'Length range after removing long and short articles: {int(np.floor(pctl[0]))} to {int(np.ceil(pctl[1]))}')

INFO: Pandarallel will run on 11 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


Length range: 0 to 236068
Number of articles after removing long and short articles: 199196
Length range after removing long and short articles: 59 to 179


In [24]:
# Remove non-English documents
df = df[df['english'] == True]
print(f'Number of articles after removing non-English: {len(df)}')

Number of articles after removing non-English: 199011


In [25]:
# Drop unneeded columns
df = df.drop(['url', 'english', 'language', 'text', 'domain', 'text_length'], axis=1)

#### Detect Irrelevant Articles

In [26]:
# Create lists of keywords to detect irrelevant articles
relevance_keywords_ai = [
    # General AI terms
    "artificial intelligence",
    "ai",
    "machine learning",
    "ml",
    "deep learning",
    "neural networks",
    "large language models",
    "llm",
    "generative",
    "gen ",
    "chatbot",
    "natural language processing",
    "nlp",
    "computer vision",
    "predictive analytics",
    "automation technology",
    "intelligent systems",
    "cognitive computing",

    # Workplace & adoption contexts
    "algorithm",
    "algorithmic",

    # Tools & platforms
    "chatgpt",
    "openai",
    "bard",
    "claude",
    "copilot",
    "grok",
    "anthropic",
    "xai",
    "autonomous",
    "self-driving",

    # Implementation areas
    "process automation",
    "business automation",
    "workflow automation",
    "robotic",
    "data-driven algorithms",
    "predictive modeling",
]

relevance_keywords_prod = [
    # Productivity & efficiency
    "productivity",
    "efficiency",
    "output",
    "performance",
    "optimization",
    "throughput",
    "automation",
    "process",
    "streamlining",
    "scalability",
    "time savings",
    "savings",
    "reduction",
    "cost",
    "automatic",
    "automation",
    "robot",
 
    # Economic impact
    "economy",
    "economic",
    "profitability",
    "competitiveness",
    "performance",
    "margins",
    "return on investment",
    "roi",
    "production",
    "scaling",
    "scale",

    # Workforce restructuring
    "work",
    "workforce",
    "job",
    "displacement",
    "reduction",
    "downsizing",
    "redundancy",
    "layoffs",
    "cuts",
    "outsourcing",
    "restructuring",
    "labor",
    "reduction",

    # Job transformation
    "reskilling",
    "upskilling",
    "redeployment",
    "retraining"
]

In [27]:
def relevance_score(text: str) -> np.ndarray:
    text = text.lower()
    patterns_ai = [re.compile(r"\b" + re.escape(k) + r"s?\b", re.I) for k in relevance_keywords_ai]
    patterns_prod = [re.compile(r"\b" + re.escape(k) + r"s?\b", re.I) for k in relevance_keywords_prod]
    score_ai = (sum(1 for p in patterns_ai if re.search(p, text)))
    score_prod = (sum(1 for p in patterns_prod if re.search(p, text)))

    return score_ai, score_prod

In [28]:
# Identify relevant articles
pandarallel.initialize(progress_bar=True)

df[['score_ai', 'score_prod']] = pd.DataFrame(df['text_clean'].parallel_apply(relevance_score).tolist(), index=df.index)

INFO: Pandarallel will run on 11 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [31]:
print(f'Number of articles with no keywords from one or both categories: {len(df[(df.score_prod == 0) | (df.score_ai == 0)])}')
print(f'Number of articles with no keywords from either categories: {len(df[(df.score_prod == 0) & (df.score_ai == 0)])}')

Number of articles with no keywords from one or both categories: 57975
Number of articles with no keywords from either categories: 8393


In [32]:
df.to_parquet('output_data/df_relevant.parquet')

In [ ]:
df = pd.read_parquet('output_data/df_relevant.parquet')

In [33]:
# Remove irrelevant articles
df = df[(df.score_prod > 0) | (df.score_ai > 0)]
print(f'Number of articles after removing irrelevant: {len(df)}')

Number of articles after removing irrelevant: 190618


In [34]:
# Create a function to get the minhash of the text
def get_minhash(text: str, num_perm: int = 128):
    # Get the minhash of the text
    m = MinHash(num_perm=num_perm)
    for token in set(text.split()):
        m.update(token.encode("utf8"))
    return m

# Deduplication through LSH
def dedupe_lsh(docs, threshold: float = 0.9, num_perm: int = 128):
    # Define an LSH object
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    minhashes = []

    # Get the minhashes for each article
    for i, doc in enumerate(docs):
        m = get_minhash(doc, num_perm=num_perm)
        lsh.insert(f"doc_{i}", m)
        minhashes.append(m)

    # Create objects to store outputs
    visited = set()
    groups = []

    # Identify articles with similar texts
    for i, m in enumerate(minhashes):
        if i in visited:
            continue
        dup_idxs = [int(j.replace("doc_", "")) for j in lsh.query(m)]
        for j in dup_idxs:
            visited.add(j)
        groups.append(sorted(dup_idxs))

    # Create ouput
    keep_mask = [True] * len(docs)
    for g in groups:
        for j in g[1:]:
            keep_mask[j] = False

    return keep_mask

In [37]:
# Identify duplicates
df['dupe'] = dedupe_lsh(df['text_clean'].tolist(), threshold=0.9)

In [38]:
print(f'Number of articles that are not duplicates: {len(df[df.dupe == True])}')

Number of articles that are not duplicates: 162130


In [39]:
# Save the deduplicated data
df = df[df.dupe == True]
df.to_parquet('output_data/df_dedupe.parquet')